In [1]:
# from pyspark.sql import SparkSession

# minio_endpoint = 'http://minio:9000'
# minio_access_key = 'admin'
# minio_secret_key = 'password'

# spark = (
#     SparkSession.builder
#     .appName("Jupyter-External-Spark")
#     .master("spark://spark-master:7077")
#     # --- Critical driver configs (for worker -> driver communication) ---
#     .config("spark.driver.host", "jupyter-spark")   # must match container name
#     .config("spark.driver.bindAddress", "0.0.0.0")
#     .config("spark.driver.port", "4041")
#     .config("spark.blockManager.port", "4042")
#     # --- S3A / MinIO configs ---
#     .config("spark.hadoop.fs.s3a.endpoint", minio_endpoint)
#     .config("spark.hadoop.fs.s3a.access.key", minio_access_key)
#     .config("spark.hadoop.fs.s3a.secret.key", minio_secret_key)
#     .config("spark.hadoop.fs.s3a.path.style.access", "true")
#     .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
#     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
#     .config("spark.hadoop.fs.s3a.aws.credentials.provider",
#             "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
#     # --- Pass credentials to executors ---
#     .config("spark.executorEnv.AWS_ACCESS_KEY_ID", minio_access_key)
#     .config("spark.executorEnv.AWS_SECRET_ACCESS_KEY", minio_secret_key)
#     .getOrCreate()
# )

# print("✅ SparkSession connected successfully!")
# print("Master:", spark.sparkContext.master)
# print("Driver host:", spark.conf.get("spark.driver.host"))


# from pyspark.sql import SparkSession

# driver_ip = "172.18.0.11"  # replace with your actual output above

# spark = (
#     SparkSession.builder
#     .appName("Jupyter-External-Spark")
#     .master("spark://spark-master:7077")
#     .config("spark.driver.bindAddress", "0.0.0.0")
#     .config("spark.driver.host", driver_ip)   # use direct IP instead of hostname
#     .config("spark.driver.port", "4041")
#     .config("spark.blockManager.port", "4042")
#     .config("spark.network.timeout", "120s")
#     .config("spark.executor.heartbeatInterval", "30s")
#     .getOrCreate()
# )

# print("✅ Connected to:", spark.sparkContext.master)
# print("Driver:", spark.conf.get("spark.driver.host"))





# 1) Force-stop any existing SparkContext (safe to run)
from pyspark import SparkContext
try:
    sc = SparkContext.getOrCreate()
    # sc may be stopped already; stop() will be no-op if already stopped
    sc.stop()
except Exception as e:
    # ignore errors while stopping
    print("stop() exception (ignored):", e)

# 2) Now create a clean SparkSession that connects to external cluster
from pyspark.sql import SparkSession
minio_endpoint = 'http://minio:9000'
minio_access_key = 'admin'
minio_secret_key = 'password'
driver_ip = "172.18.0.11"   # e.g. '172.18.0.5' or 'jupyter-spark'

spark = (
    SparkSession.builder
    .appName("Jupyter-External-Spark")
    .master("spark://spark-master:7077")
    .config("spark.driver.bindAddress", "0.0.0.0")
    .config("spark.driver.host", driver_ip)
    .config("spark.driver.port", "4041")
    .config("spark.blockManager.port", "4042")
    .config("spark.network.timeout", "120s")
    .config("spark.executor.heartbeatInterval", "30s")
    # MinIO / S3A
    .config("spark.hadoop.fs.s3a.endpoint", minio_endpoint)
    .config("spark.hadoop.fs.s3a.access.key", minio_access_key)
    .config("spark.hadoop.fs.s3a.secret.key", minio_secret_key)
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    # ensure executors have creds
    .config("spark.executorEnv.AWS_ACCESS_KEY_ID", minio_access_key)
    .config("spark.executorEnv.AWS_SECRET_ACCESS_KEY", minio_secret_key)
    .getOrCreate()
)

print("Spark master:", spark.sparkContext.master)
print("Driver:", spark.conf.get("spark.driver.host"))




Spark master: spark://spark-master:7077
Driver: 172.18.0.11


In [2]:
spark.range(0, 10).show()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# dataframe = spark.read.json('s3a://test/orders.json')

# average = dataframe.agg({'amount':'avg'})

# #average.write.format("csv").option("header", "true").save("s3a://test/json/")

# average.show()